In [1]:
import numpy as np
import torch
from torch.utils import data

In [2]:
def synthetic_data(w, b, num_examples): 
    """Generate y = Xw + b + noise."""
    X = torch.normal(0, 1, (num_examples, len(w))) # Generate X
    y = torch.matmul(X, w) + b  # calculate y from X
    y += torch.normal(0, 0.01, y.shape) # noise of normal distribution of std 0.01
    return X, y.reshape((-1, 1))

In [3]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

### Reading Dataset

In [4]:
def load_array(data_arrays, batch_size, is_train=True): #@save
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [5]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [6]:
from torch import nn
net = nn.Sequential(nn.Linear(2, 1))

In [7]:
# Intialize model Parameters
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [8]:
## Define Loss function
loss = nn.MSELoss()

In [9]:
# Optimization Algorithm
trainer = torch.optim.SGD(net.parameters(),lr = 0.03)

### Training 

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)                         # calculate y_pred and the loss
        trainer.zero_grad()
        l.backward()
        trainer.step()                              # update parameters
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000202
epoch 2, loss 0.000108
epoch 3, loss 0.000109


In [11]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([ 0.0001, -0.0005])
error in estimating b: tensor([-0.0009])
